In [19]:
import pandas as pd
import numpy as np
import sys
def drop_duplicates(df,column='z'):
    tmp=df.copy()
    tmp['zstr']=tmp[column].astype(str)#.drop_duplicates(subset='z')
    tmp=tmp.drop_duplicates(subset='zstr')
    return tmp.drop('zstr',axis='columns')

In [20]:
REBUILD=False

In [21]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn=pd.read_json('s9.json')

dfn['n']=dfn['z'].apply(lambda l: 8 if l.count(0)==1 else 9)

if dfn[dfn['z'].apply(lambda l: l.count(0))>1].empty:
    print('Ready n')
    kk=dfn['z'].apply(lambda l: l.remove(0) if l.count(0)==1 else l )

assert dfn.shape[0]==dfn['z'].astype(str).unique().shape[0]

for n in [8,7,6]:
    pdfn={}
    pdfn[n]=pd.read_json('s{}.json'.format(n))

    assert pdfn[n].shape[0]==pdfn[n]['z'].astype(str).unique().shape[0]

    pdfn[n]['n']=pdfn[n]['z'].apply(lambda l: n-1 if l.count(0)==1 else n)

    if pdfn[n][pdfn[n]['z'].apply(lambda l: l.count(0))>1].empty:
        print('Ready n')
        kk=pdfn[n]['z'].apply(lambda l: l.remove(0) if l.count(0)==1 else l )
    print(n,'→',pdfn[n][pdfn[n]['n']==n].shape)
    dfn=dfn.append(pdfn[n]).reset_index(drop=True)
    
    #Reordering:
    dfn['ls']=dfn['l'].apply(lambda l: np.abs(l).max())
    dfn['ks']=dfn['k'].apply(lambda l: np.abs(l).max())
    dfn=dfn.sort_values(['ls','ks']).reset_index(drop=True)
    dfn=dfn.drop(['ls','ks'],axis='columns')
    #dfn=dfn.sort_values(['gcd']).reset_index(drop=True)
    dfn=drop_duplicates(dfn,column='z')
    print(n,'→',dfn[dfn['n']==n].shape)

SystemExit: Ignore this cell

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn[dfn['z'].apply(lambda l: l.count(0)>0)]

SystemExit: Ignore this cell

In [23]:
if not REBUILD:
    sys.exit('Ignore this cell')
rng={}
for n in [6,7,8,9]:
    rng[n]=[(dfn[dfn['n']==n].l.apply(lambda l: max(l)).max(),dfn[dfn['n']==n].l.apply(lambda l: min(l)).min()),
 (dfn[dfn['n']==n].k.apply(lambda l: max(l)).max(),dfn[dfn['n']==n].k.apply(lambda l: min(l)).min())]

SystemExit: Ignore this cell

`
{6: [(21, -21), (21, -21)],
  7: [(16, -16), (16, -16)],
  8: [(15, -15), (15, -15)],
  9: [(12, -12), (12, -12)]}
`

In [24]:
if not REBUILD:
    sys.exit('Ignore this cell')
rng

SystemExit: Ignore this cell

In [25]:
if not REBUILD:
    sys.exit('Ignore this cell')
dfn=dfn.rename(columns={'z':'solution'})
dfn.to_json('solutions.json',orient='records')

SystemExit: Ignore this cell

In [26]:
import os
if os.path.exists('/content'):
    !git clone https://github.com/restrepo/anomaly.git 2> /dev/null > /dev/null
    ! pip install anomalies 2> /dev/null > /dev/null
    os.chdir('anomaly')

Inehirate from `solutions` class to explore for solutions with repeated charges

## Find solutions with repeated charges

In [27]:
ds=pd.read_json('solutions.json')

In [28]:
ds.shape

(31211, 5)

In [29]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(31211, 6)

In [30]:
ds=ds.drop('strz',axis='columns')

In [31]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

16805

## Check phenomenological conditions
$$ 2l+\nu-m=0$$

In [32]:
#See anomalysolutions.ipynb for details
from anomalytools import *

## Filter solutions with at most two massless chiral fermions

DD analysis

In [329]:
#cl['sltn']=cl['solution'].astype(str)
#cl['sltn']=cl['sltn'].apply(jsonKeys2int)
#cl=cl.sort_values('n').reset_index(drop=True).drop('sltn',axis='columns')

In [330]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    for x in ll:
        if ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2:
        MR=True
    return MR

In [332]:
def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level(ll):
    '''
    Given a solution find set of repeated charges and assign to `l` and `ν`, e.g
       ll=[l,l,...,ν,ν,...]
    For each combination of repeated sets calculates:
       m=2l+ν
    If `m ≠ l ≠ ν` in ll, returns a `N → N+4` solution
       [m,m,m,-m-m,l,l,...,ν,ν,...]
    else, returns a `N → N+6` solution
       [m,m,m,-m,-m,-m,l,l,...,ν,ν,...]
    '''
    ms=[]
    rp=[]
    for x in ll:
        if ll.count(x)==1:
            ms.append(x)
        elif ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    #print(rp,isinstance(rp,list))
    #Two or more repeated sets
    sltn=[]
    if len(rp)>=2:
        rps=[k  for k  in itertools.combinations(rp,2)]
        #print('rps1',rps)
        rps=rps+[tuple(reversed(k)) for k  in rps]
        #print('rps2',rps)
        for r in rps:
            #Loop on combinatories here, break whit the first solution
            l=r[0]
            ν=r[1]
            m=2*l+ν
            s=l-m
            massless=ms.copy()
            massless=massless+[x for x in rp if x not in r and x not in massless]
            #print('ini massless',massless)
            if m in ll and m not in rp:
                massless.remove(m)
                #Check Dirac and Majorana
                massless=extract_Dirac_and_Majorana(massless,s)
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,m,0],'massless':massless}
            else:
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,0,m],'massless':massless}
                
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)
            sltn.append(slt)
    return sltn

In [366]:
assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
assert best_higgs_singlet([1,-11,9])==(2, [])

assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]

assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

In [409]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].reset_index(drop=True)

In [410]:
cl['tree_level']=cl['solution'].apply(tree_level)

In [411]:
pd.set_option('display.max_colwidth',400)
#cl['tree_level']

In [412]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

In [413]:
cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5
1,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,"[{'[l,ν,s,m_in,m_out]': [-9, 1, 8, 0, -17], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -9, 8, 0, -7], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}]",11
2,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,"[{'[l,ν,s,m_in,m_out]': [-4, 9, -5, 1, 0], 'massless': [-11], 'sp': 22, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [9, -4, -5, 0, 14], 'massless': [1, -11], 'sp': 10, 'sp_massless': []}]",11
3,"[0, 1]","[-1, -4, 1]","[1, 1, 11, -16, -16, 19]",2,6,"[{'[l,ν,s,m_in,m_out]': [-16, 1, 15, 0, -31], 'massless': [11, 19], 'sp': 30, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -16, 15, 0, -14], 'massless': [11, 19], 'sp': 30, 'sp_massless': []}]",19
4,"[-3, 3]","[-3, -4]","[5, -9, -9, 16, 16, -19]",18,6,"[{'[l,ν,s,m_in,m_out]': [-9, 16, -7, 0, -2], 'massless': [5, -19], 'sp': 14, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [16, -9, -7, 0, 23], 'massless': [5, -19], 'sp': 14, 'sp_massless': []}]",19
...,...,...,...,...,...,...,...
349,"[6, 3, -3]","[-2, -5, 2, -3]","[5, 6, 6, -11, -11, -16, 25, 26, -30]",28,9,"[{'[l,ν,s,m_in,m_out]': [-11, 6, 5, -16, 0], 'massless': [5, 26], 'sp': 31, 'sp_massless': []}]",30
350,"[6, 0, 4]","[-3, -1, -7, 9]","[2, -3, -6, -6, 18, 18, -24, -29, 30]",30,9,"[{'[l,ν,s,m_in,m_out]': [18, -6, -12, 30, 0], 'massless': [2, -3, -24, -29], 'sp': 27, 'sp_massless': []}]",30
351,"[9, -1, 2]","[-1, -2, -1, -3]","[9, 9, 9, -12, -19, -19, 22, -29, 30]",8,9,"[{'[l,ν,s,m_in,m_out]': [-19, 9, 10, -29, 0], 'massless': [30], 'sp': 60, 'sp_massless': []}]",30
352,"[-5, 9, 1]","[-2, -1, 4, 2]","[4, -5, -10, -10, -11, 14, 24, 24, -30]",4,9,"[{'[l,ν,s,m_in,m_out]': [-10, 24, -14, 4, 0], 'massless': [-5, -11, 14, -30], 'sp': 16, 'sp_massless': []}]",30
